<center>
<h1 style="font-family:verdana">
 💻 🧑 Classificació d'intencions 🧑 💻


### ESTUDI COMPARATIU ENTRE LEMMATIZER I STEMMER
A l'arxiu, la implementació de *stemming* i lematització s'ha dut a terme mitjançant l'ús de les llibreries de NLTK. Per a *stemming*, s'ha emprat el *PorterStemmer*, el qual processa cada paraula d'una oració, reduint-la a la seva arrel. La funció `stem_sentence` realitza aquest procés: pren una oració, la tokeniza i aplica *stemming* a cada paraula, per després unir-les novament en una oració completa. Aquest procediment s'aplica a totes les oracions del conjunt d'entrenament, generant un nou conjunt (train_sentences_stem) que, posteriorment, és convertit en seqüències numèriques mitjançant un Tokenizer. Aquest `Tokenizer` construeix un vocabulari específic basat en les paraules resultants del *stemming*, i les seqüències resultants s'ajusten a una longitud uniforme usant `pad_sequences`.

Per a la lematització, s'ha utilitzat `WordNetLemmatizer` de NLTK, que transforma cada paraula cap a la seva forma base considerant el seu context. La funció `lemmatize_text` processa les oracions de manera similar al *stemming*, aplicant lematització a cada paraula i unint els resultats en una oració. Aquest conjunt de frases lematitzades (`train_sentences_lem`) també es converteix en seqüències numèriques amb un `Tokenizer`, generant un altre vocabulari específic per a les paraules lematitzades. Igual que amb *stemming*, les seqüències lematitzades s'adapten amb `pad_sequences` per a un processament uniforme. Ambdues versions (amb *stemming* i lematització) s'avaluen finalment als conjunts de validació i prova, permetent analitzar l'impacte de cada tècnica en els resultats del model.

In [35]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D, Dropout, Conv1D, GlobalAveragePooling1D, LayerNormalization #Remove
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer



# Descargamos los recursos necesarios para la lematización
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

<h1><a name="section-one"> 1. Inspecció del conjunt de dades </a></h1>

A la carpeta `data` tenim els diferents fitxers CSV que utilitzarem per a aquesta pràctica.

En primer lloc, llegirem les dades dels fitxers CSV amb `pandas`.

In [36]:
train_data = pd.read_csv('./data/train.csv', header=None) # la última columna té la intenció de cada frase (1a columna)
val_data = train_data.tail(900) # Partim una part del train per tenir validation partition
train_data = pd.read_csv('./data/train.csv', header=None, nrows=4078)
test_data = pd.read_csv('./data/test.csv', header=None)

In [37]:
random_number = random.randint(0, len(train_data)-1)

train_sentences = list(train_data[0]) # primera columna del dataset, té la frase del usuari
train_labels = list(s.replace('"', '') for s in train_data[2]) 
train_labels = list(s.replace(' ', '') for s in train_labels)

In [38]:
num_labels = 0  # Les labels son les possibles intencions del usuari
for label in set(train_labels):
  print(f'Label {num_labels}:', label.split('.')[-1])
  num_labels += 1

Label 0: abbreviation
Label 1: airline
Label 2: cheapest
Label 3: flight+airfare
Label 4: ground_service
Label 5: ground_fare
Label 6: flight_no
Label 7: city
Label 8: aircraft+flight+flight_no
Label 9: airfare
Label 10: restriction
Label 11: meal
Label 12: aircraft
Label 13: quantity
Label 14: distance
Label 15: airline+flight_no
Label 16: airfare+flight_time
Label 17: ground_service+ground_fare
Label 18: flight_time
Label 19: flight
Label 20: capacity
Label 21: airport


<h1><a name="section-two"> 2. Preprocessament de dades </a></h1>

En primer lloc, haurem de tokenitzar les oracions. Això consisteix a convertir el text en representacions numèriques, ja que els models esperen unitats discretes.

En aquesta pràctica farem servir una tokenització senzilla, simplement dividirem les oracions en paraules i crearem un vocabulari basat en les paraules úniques de les dades d'entrenament. Cada paraula (token) tindrà assignat un ID únic.

Vegem com queda el vocabulari.

In [39]:
num_words=500 
# Convertimos todas las frases a minúsculas
train_sentences_lower = [sentence.lower() for sentence in train_sentences]  # MOD

tokenizer = Tokenizer(num_words) 
tokenizer.fit_on_texts(train_sentences)

# Tokenitzem perquè cal codificar en int les paraules
vocab = tokenizer.word_index
print(vocab)  # Ens queden les consultes del usuari tokenitzades així podem utilitzar els tokens per buscar INTENT
print(len(vocab))

{'to': 1, 'from': 2, 'flights': 3, 'the': 4, 'on': 5, 'what': 6, 'me': 7, 'flight': 8, 'boston': 9, 'show': 10, 'san': 11, 'i': 12, 'denver': 13, 'a': 14, 'francisco': 15, 'in': 16, 'and': 17, 'atlanta': 18, 'pittsburgh': 19, 'is': 20, 'dallas': 21, 'baltimore': 22, 'all': 23, 'philadelphia': 24, 'like': 25, 'are': 26, 'list': 27, 'airlines': 28, 'of': 29, 'between': 30, 'that': 31, 'washington': 32, 'leaving': 33, 'please': 34, 'pm': 35, 'morning': 36, 'would': 37, 'fly': 38, 'for': 39, 'fare': 40, 'first': 41, 'wednesday': 42, 'after': 43, 'there': 44, 'oakland': 45, "'d": 46, 'ground': 47, 'you': 48, 'does': 49, 'trip': 50, 'transportation': 51, 'class': 52, 'arriving': 53, 'cheapest': 54, 'need': 55, 'city': 56, 'round': 57, 'with': 58, 'before': 59, 'which': 60, 'available': 61, 'have': 62, 'give': 63, 'at': 64, 'fares': 65, 'american': 66, 'afternoon': 67, 'one': 68, 'want': 69, 'how': 70, 'way': 71, 'new': 72, 'dc': 73, 'nonstop': 74, 'arrive': 75, 'earliest': 76, 'york': 77, 'g

In [40]:
stemmer = PorterStemmer()

# Definir la función de stemming
def stem_sentence(sentence):
    words = nltk.word_tokenize(sentence)  # Tokenizar la oración
    stemmed_words = [stemmer.stem(word) for word in words]  # Aplicar stemming
    return " ".join(stemmed_words)

train_sentences_stem = [stem_sentence(sentence) for sentence in train_sentences]

stem_tokenizer = Tokenizer(num_words)
stem_tokenizer.fit_on_texts(train_sentences_stem)

stem_vocab = stem_tokenizer.word_index
print(stem_vocab)
print(len(stem_vocab))

{'to': 1, 'from': 2, 'flight': 3, 'the': 4, 'on': 5, 'what': 6, 'me': 7, 'boston': 8, 'show': 9, 'san': 10, 'i': 11, 'denver': 12, 'a': 13, 'francisco': 14, 'in': 15, 'and': 16, 'atlanta': 17, 'pittsburgh': 18, 'is': 19, 'dalla': 20, 'baltimor': 21, 'all': 22, 'philadelphia': 23, 'airlin': 24, 'leav': 25, 'like': 26, 'are': 27, 'list': 28, 'fare': 29, 'arriv': 30, 'of': 31, 'between': 32, 'that': 33, 'washington': 34, 'fli': 35, 'pleas': 36, 'morn': 37, 'pm': 38, 'would': 39, 'for': 40, 'wednesday': 41, 'first': 42, 'after': 43, 'there': 44, 'oakland': 45, "'d": 46, 'citi': 47, 'ground': 48, 'you': 49, 'transport': 50, 'doe': 51, 'class': 52, 'trip': 53, 'go': 54, 'cheapest': 55, 'need': 56, 'round': 57, 'with': 58, 'befor': 59, 'which': 60, 'avail': 61, 'have': 62, 'give': 63, 'at': 64, 'american': 65, 'afternoon': 66, 'one': 67, 'want': 68, 'airport': 69, 'how': 70, 'way': 71, 'new': 72, 'nonstop': 73, 'dc': 74, 'thursday': 75, 'stop': 76, 'tuesday': 77, 'monday': 78, 'earliest': 79,

In [41]:
# LEMMATIZER
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

# Lematización de las frases
train_sentences_lem = [lemmatize_text(sentence) for sentence in train_sentences]

# Tokenizer para convertir las frases lematizadas en secuencias
lem_tokenizer = Tokenizer()
lem_tokenizer.fit_on_texts(train_sentences_lem)

lem_vocab = lem_tokenizer.word_index
print(lem_vocab)
print(len(lem_vocab))

{'to': 1, 'from': 2, 'flight': 3, 'the': 4, 'on': 5, 'what': 6, 'me': 7, 'boston': 8, 'show': 9, 'san': 10, 'i': 11, 'denver': 12, 'a': 13, 'francisco': 14, 'in': 15, 'and': 16, 'atlanta': 17, 'pittsburgh': 18, 'is': 19, 'dallas': 20, 'baltimore': 21, 'all': 22, 'philadelphia': 23, 'airline': 24, 'like': 25, 'are': 26, 'list': 27, 'fare': 28, 'of': 29, 'between': 30, 'that': 31, 'washington': 32, 'leaving': 33, 'please': 34, 'morning': 35, 'fly': 36, 'pm': 37, 'would': 38, 'for': 39, 'wednesday': 40, 'first': 41, 'after': 42, 'there': 43, 'oakland': 44, "'d": 45, 'city': 46, 'ground': 47, 'you': 48, 'doe': 49, 'class': 50, 'trip': 51, 'transportation': 52, 'arriving': 53, 'cheapest': 54, 'need': 55, 'round': 56, 'with': 57, 'before': 58, 'which': 59, 'available': 60, 'have': 61, 'give': 62, 'at': 63, 'american': 64, 'go': 65, 'afternoon': 66, 'one': 67, 'want': 68, 'airport': 69, 'how': 70, 'way': 71, 'new': 72, 'nonstop': 73, 'dc': 74, 'thursday': 75, 'arrive': 76, 'tuesday': 77, 'mon

---

 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>

En aquest primer exercici us demanem que donat el vocabulari anterior convertiu la llista d'oracions de la partició d'entrenament, és a dir, `train_sentenes` en seqüències d'IDs.

Podeu trobar la documentació [aquí](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer).

In [42]:
#TODO: la manera en que estem tenint en compte el vocabulari e:
# cada paraula te un index, i ens resulta en els index assignats a cada paraula (fixat en que to es 1)

train_sequences = tokenizer.texts_to_sequences(train_sentences)

print(train_sentences[0])
print(train_sequences[0])

i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
[12, 69, 1, 38, 2, 9, 64, 415, 84, 17, 75, 16, 13, 64, 493, 16, 4, 36]


Si ho heu fet correctament hauríeu d'obtenir això:

```
print(train_sentences[0])
print(train_sequences[0])

i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
[12, 69, 1, 38, 2, 9, 64, 415, 84, 17, 75, 16, 13, 64, 493, 16, 4, 36]
```



In [43]:
train_sequences_stem = stem_tokenizer.texts_to_sequences(train_sentences_stem)

print(train_sentences_stem[0])
print(train_sequences_stem[0])

i want to fli from boston at 838 am and arriv in denver at 1110 in the morn
[11, 68, 1, 35, 2, 8, 64, 377, 82, 16, 30, 15, 12, 64, 441, 15, 4, 37]


In [44]:
train_sequences_lem = lem_tokenizer.texts_to_sequences(train_sentences_lem)

print(train_sentences_lem[0])
print(train_sequences_lem[0])

# com que el vocab s'ordena per freqüència, ha canviat el index de les paraules

i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
[11, 68, 1, 36, 2, 8, 63, 394, 83, 16, 76, 15, 12, 63, 462, 15, 4, 35]


---
A continuació haurem d'aconseguir que totes les seqüències tinguen una longitud fixa. Per a fer això primer fixarem la longitud segons la longitud màxima trobada a les seqüències del conjunt d'entrenament. I a continuació omplirem (*pad*) les seqüències que tinguen una longitud menor.


In [45]:
max_sequence_length = max(map(len, train_sequences))
train_pad_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
print('Padded sequence: ', train_pad_sequences[0]) 
# D'aquesta manera assegurem que totes les sequencies siguin iguals posant padding --> APORTA SOROLL
# La millor manera de reduir el soroll es tenirlo darrere en comptes de davant

Padded sequence:  [ 12  69   1  38   2   9  64 415  84  17  75  16  13  64 493  16   4  36
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]


In [46]:
# padding STEM
max_sequence_length_stem = max(map(len, train_sequences_stem))
train_pad_sequences_stem = pad_sequences(train_sequences_stem, maxlen=max_sequence_length_stem, padding='post')
print('Padded sequence: ', train_pad_sequences_stem[0]) 

Padded sequence:  [ 11  68   1  35   2   8  64 377  82  16  30  15  12  64 441  15   4  37
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]


In [47]:
# padding LEM 
max_sequence_length_lem = max(map(len, train_sequences_lem))
train_pad_sequences_lem = pad_sequences(train_sequences_lem, maxlen=max_sequence_length_lem, padding='post')
print('Padded sequence: ', train_pad_sequences_lem[0]) 

Padded sequence:  [ 11  68   1  36   2   8  63 394  83  16  76  15  12  63 462  15   4  35
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


---

### One-Hot Encoding
A continuació convertirem les classes d'intencions categòriques (*capacity*, *ground_service*, *flight*, etc.) en el que anomenem one-hot vector encoding. Aquesta tècnica s'utilitza per representar les dades categòriques com a vectors binaris. On cada vector representa una classe específica i l'element corresponent a la classe es posa a 1 i la resta d'elements es mantenen a 0.

Imaginem que tenim tres classes: *capacity*, *ground_service*, *flight*. Podríem codificar aquestes classes amb un vector únic de la forma següent:


```
   capacity -> [1, 0, 0]
   ground_service -> [0, 1, 0]
   flight -> [0, 0, 1]
```

Per aconseguir això primer codificarem les classes d'intenció en etiquetes numèriques.

In [48]:
# Això ja no te a veure amb el vocab i no el tokenitzador: no hem de fer 3 versions

label_encoder = LabelEncoder()
train_numerical_labels = label_encoder.fit_transform(train_labels)

# Estem fent One-hot encoding dels labels osigui de les intencions que l'usuari pot tenir
print(f'Original labels: {train_labels}\n')
print(f'Encoded labels: {train_numerical_labels} \n')

Original labels: ['flight', 'flight', 'flight_time', 'airfare', 'airfare', 'flight', 'aircraft', 'flight', 'flight', 'ground_service', 'flight', 'flight', 'airport', 'flight', 'flight', 'airfare', 'ground_service', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'aircraft', 'airfare', 'flight', 'airline', 'flight', 'ground_service', 'flight', 'airfare', 'flight', 'flight', 'flight', 'flight', 'airfare', 'airline', 'flight', 'flight', 'flight', 'distance', 'flight', 'airline', 'airline', 'flight', 'airline', 'ground_service', 'abbreviation', 'flight', 'flight', 'flight_time', 'flight', 'flight', 'ground_fare', 'flight', 'abbreviation', 'flight', 'flight', 'flight', 'flight', 'flight', 'airline', 'flight', 'ground_service', 'airline', 'flight', 'flight', 'airport', 'flight', 'flight', 'abbreviation', 'flight', 'flight', 'flight', 'flight', 'aircraft', 'airfare', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'airline', 'flight', 'flight', 'flight', 'fli

I a continuació convertim les etiquetes a vectors one-hot:

In [49]:
num_classes = len(np.unique(train_numerical_labels))
train_encoded_labels = to_categorical(train_numerical_labels, num_classes)

print('Example: \n')
print(f'Original label: {train_labels[0]}\n')
print(f'Numerical label: {train_numerical_labels[0]}\n')
print(f'One-hot: {train_encoded_labels[0]}\n')

Example: 

Original label: flight

Numerical label: 12

One-hot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



Veiem que hi ha un 1 allà on el índex coincideix amb l'etiqueta. 

Lemmatize

In [50]:
# Això ja no te a veure amb el vocab i no el tokenitzador: no hem de fer 3 versions

label_encoder = LabelEncoder()
train_numerical_labels = label_encoder.fit_transform(train_labels)

# Estem fent One-hot encoding dels labels osigui de les intencions que l'usuari pot tenir
print(f'Original labels: {train_labels}\n')
print(f'Encoded labels: {train_numerical_labels} \n')

Original labels: ['flight', 'flight', 'flight_time', 'airfare', 'airfare', 'flight', 'aircraft', 'flight', 'flight', 'ground_service', 'flight', 'flight', 'airport', 'flight', 'flight', 'airfare', 'ground_service', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'aircraft', 'airfare', 'flight', 'airline', 'flight', 'ground_service', 'flight', 'airfare', 'flight', 'flight', 'flight', 'flight', 'airfare', 'airline', 'flight', 'flight', 'flight', 'distance', 'flight', 'airline', 'airline', 'flight', 'airline', 'ground_service', 'abbreviation', 'flight', 'flight', 'flight_time', 'flight', 'flight', 'ground_fare', 'flight', 'abbreviation', 'flight', 'flight', 'flight', 'flight', 'flight', 'airline', 'flight', 'ground_service', 'airline', 'flight', 'flight', 'airport', 'flight', 'flight', 'abbreviation', 'flight', 'flight', 'flight', 'flight', 'aircraft', 'airfare', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'airline', 'flight', 'flight', 'flight', 'fli

In [51]:
num_classes_lem = len(np.unique(train_numerical_labels))
train_encoded_labels = to_categorical(train_numerical_labels, num_classes)

print('Example: \n')
print(f'Original label: {train_labels[0]}\n')
print(f'Numerical label: {train_numerical_labels[0]}\n')
print(f'One-hot: {train_encoded_labels[0]}\n')

Example: 

Original label: flight

Numerical label: 12

One-hot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



---

 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏</a></h1>

Amb la partició de validació i test haurem de realitzar els mateixos passos. Per tant, en aquest exercici us demanem que obtingueu `val_pad_sequences`, `val_encoded_labels`, `test_pad_sequences` i `test_encoded_labels`.

In [52]:
# PER A LES NORMALS
val_sentences = list(val_data[0])
test_sentences = list(test_data[0])

val_sequences = tokenizer.texts_to_sequences(val_sentences) # no cal definir vocabulari evidentment, perque es el mateix
test_sequences = tokenizer.texts_to_sequences(test_sentences)

val_pad_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length, padding='post')
test_pad_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

In [53]:
# PER A LES STEMMED

# Aplicar el stemming a todas las oraciones de validación y prueba (val y test)
val_sentences_stem = [stem_sentence(sentence) for sentence in val_sentences]
test_sentences_stem = [stem_sentence(sentence) for sentence in test_sentences]

# PER A LES STEMMED (Val y Test)
val_sequences_stem = stem_tokenizer.texts_to_sequences(val_sentences_stem)  # Aplicar stemming y convertir a secuencias
test_sequences_stem = stem_tokenizer.texts_to_sequences(test_sentences_stem)  # Hacer lo mismo con el test

# Padding de las secuencias para validación y test
val_pad_sequences_stem = pad_sequences(val_sequences_stem, maxlen=max_sequence_length_stem, padding='post')
test_pad_sequences_stem = pad_sequences(test_sequences_stem, maxlen=max_sequence_length_stem, padding='post')


In [54]:
# PER A LES LEMMATIZED

# Aplicar la lematización a todas las oraciones de validación y prueba (val y test)
val_sentences_lem = [lemmatize_text(sentence) for sentence in val_sentences]
test_sentences_lem = [lemmatize_text(sentence) for sentence in test_sentences]

# PER A LES LEMMATIZED (Val y Test)
val_sequences_lem = lem_tokenizer.texts_to_sequences(val_sentences_lem)  # Aplicar lematización y convertir a secuencias
test_sequences_lem = lem_tokenizer.texts_to_sequences(test_sentences_lem)  # Hacer lo mismo con el test

# Padding de las secuencias para validación y test
val_pad_sequences_lem = pad_sequences(val_sequences_lem, maxlen=max_sequence_length, padding='post')
test_pad_sequences_lem = pad_sequences(test_sequences_lem, maxlen=max_sequence_length, padding='post')

In [55]:
# Les etiquetes de moment són iguals per norm, lem i stem 
val_labels = list(val_data[2])
val_labels = list(s.replace('"', '') for s in val_data[2]) 
val_labels = list(s.replace(' ', '') for s in val_labels)

test_labels = list(test_data[2])
test_labels = list(s.replace('"', '') for s in test_data[2]) 
test_labels = list(s.replace(' ', '') for s in test_labels)

In [56]:
def remove_values_and_indices(input_list, values_to_remove, other_list):
    indices_to_remove = [idx for idx, item in enumerate(input_list) if item in values_to_remove]
    cleaned_input_list = [item for idx, item in enumerate(input_list) if idx not in indices_to_remove]
    cleaned_other_list = [item for idx, item in enumerate(other_list) if idx not in indices_to_remove]
    return cleaned_input_list, np.array(cleaned_other_list)

# Remover valores no deseados
values_to_remove = ['day_name', 'airfare+flight', 'flight+airline', 'flight_no+airline']

In [57]:
# STEMM --> comentar si colem que funcioni amb lem

test_labels, test_pad_sequences_stem = remove_values_and_indices(test_labels, values_to_remove, test_pad_sequences_stem)

# Asegúrate de que ahora ambas listas tengan la misma longitud
assert len(test_labels) == len(test_pad_sequences_stem), "La longitud de test_labels y test_pad_sequences_lem no coincide"

# Convertir las etiquetas de test a valores numéricos y one-hot encoding
test_numerical_labels = label_encoder.transform(test_labels)
test_encoded_labels = to_categorical(test_numerical_labels, num_classes)

# Asegúrate de que las dimensiones coincidan
assert test_pad_sequences_stem.shape[0] == test_encoded_labels.shape[0], "Las formas de test_pad_sequences_lem y test_encoded_labels no coinciden"

print(test_pad_sequences_stem.shape)
print(test_encoded_labels.shape)

(888, 45)
(888, 22)


In [58]:
# # LEMM --> comentar si colem que funcioni amb stem

# test_labels, test_pad_sequences_lem = remove_values_and_indices(test_labels, values_to_remove, test_pad_sequences_lem)

# # Asegúrate de que ahora ambas listas tengan la misma longitud
# assert len(test_labels) == len(test_pad_sequences_lem), "La longitud de test_labels y test_pad_sequences_lem no coincide"

# # Convertir las etiquetas de test a valores numéricos y one-hot encoding
# test_numerical_labels = label_encoder.transform(test_labels)
# test_encoded_labels = to_categorical(test_numerical_labels, num_classes)

# # Asegúrate de que las dimensiones coincidan
# assert test_pad_sequences_lem.shape[0] == test_encoded_labels.shape[0], "Las formas de test_pad_sequences_lem y test_encoded_labels no coinciden"

# print(test_pad_sequences_lem.shape)
# print(test_encoded_labels.shape)

In [59]:
# normal 
val_labels, val_pad_sequences = remove_values_and_indices(val_labels, values_to_remove, val_pad_sequences)
test_labels, test_pad_sequences = remove_values_and_indices(test_labels, values_to_remove, test_pad_sequences)

In [60]:
#stem
val_labels_stem, val_pad_sequences_stem = remove_values_and_indices(val_labels, values_to_remove, val_pad_sequences_stem)
test_labels_stem, test_pad_sequences_stem = remove_values_and_indices(test_labels, values_to_remove, test_pad_sequences_stem)

In [61]:
#lem
val_labels_lem, val_pad_sequences_lem = remove_values_and_indices(val_labels, values_to_remove, val_pad_sequences_lem)
test_labels_lem, test_pad_sequences_lem = remove_values_and_indices(test_labels, values_to_remove, test_pad_sequences_lem)

In [62]:
val_numerical_labels = label_encoder.transform(val_labels_lem)
val_encoded_labels = to_categorical(val_numerical_labels, num_classes)

test_numerical_labels = label_encoder.transform(test_labels_lem)
test_encoded_labels = to_categorical(test_numerical_labels, num_classes)

In [63]:
print('Example train: \n')
print(f'Original label: {val_labels[0]}\n')
print(f'Numerical label: {val_numerical_labels[0]}\n')
print(f'One-hot: {val_encoded_labels[0]}\n')

print('Example test: \n')
print(f'Original label: {test_labels[0]}\n')
print(f'Numerical label: {test_numerical_labels[0]}\n')
print(f'One-hot: {test_encoded_labels[0]}\n')

Example train: 

Original label: flight

Numerical label: 12

One-hot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Example test: 

Original label: flight

Numerical label: 12

One-hot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



---

<h1><a name="section-three"> 3. Disseny del model i entrenament </a></h1>

En primer lloc, anem a comprovar si hi ha GPUs disponibles. A continuació si hi ha GPUs disponibles el codi assegurarà que *TensorFlow* només assigne memòria GPU quan siga necessari.

In [64]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available!")
else:
    print("GPU is not available. The model will be trained on CPU.")

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

GPU is not available. The model will be trained on CPU.


---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏</a></h1>

En aquest exercici haureu de dissenyar l'arquitectura del model. El nostre model tindrà quatre capes:

1. La primera capa serà un **embedding**. Aquesta capa permetrà convertir les dades de text d'entrada, en vectors densos amb una mida fixa (*embedding_dim*). Aquesta representació més compacta permetrà per una part capturar la informació semàntica del text d'entrada, permetent així generalitzar millor i comprendre les relacions entre les paraules. I, per una altra banda, reduir la complexitat computacional, accelerant així el temps d'entrenament i inferència. En resum, aquesta capa assignarà a cada índex de cada paraula un vector dens de mida *embedding_dim*.

2. La segona capa serà un **pooling** layer. L'entrada d'aquesta capa serà un tensor 3D (*batch_size*, *sequence_length*, *embedding_dim*). Aquesta capa es centrarà a capturar la informació més important de la seqüència d'entrada, és a dir, prendrà el valor màxim de la seqüència, donant lloc a un tensor 2D (batch_size, embedding_dim).

3. La tercera capa serà una capa **densa**. És a dir, una capa completament connectada (*fully-connected*): cada neurona d'aquesta capa estarà connectada a totes les neurones de la capa anterior. La funció d'activació que utilitzarem serà una ReLU. Aquesta funció introdueix una no-linealitat al model permetent així aprendre relacions complexes en les dades.

4. L'última capa també serà una capa **densa**. En aquest cas la funció d'activació haurà de ser la funció Softmax. Aquesta funció es fa servir per a convertir els valors de la capa anterior (*logits*) en probabilitats normalitzades. El valor de cada element de sortida representarà la probabilitat que l'entrada pertanya a una classe específica.


📢  Les capes que haureu de fer servir les podreu trobar [aquí](https://www.tensorflow.org/api_docs/python/tf/keras/layers).





In [65]:
# # NORMAL 
# embedding_dim = 100

# vocab_size = num_words + 1

# model = Sequential()
# model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length)) #layer 1 - embedding 
# model.add(GlobalMaxPooling1D()) #layer 2 - pooling
# model.add(Dense(64, activation='relu')) #layer 3 -relu
# model.add(Dense(num_classes, activation='softmax')) #layer 4 - softmax

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# batch_size = 32
# epochs = 2

# model.fit(train_pad_sequences, train_encoded_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences, val_encoded_labels))

# # Evaluate the model on the test set
# loss, accuracy = model.evaluate(test_pad_sequences, test_encoded_labels, batch_size=batch_size)
# print(f"Test accuracy: {accuracy:.2f}")

El resultat d'aquest model és una accuracy de 0.82. 

In [66]:
# Example function to simulate the printing of key data shapes
def check_data_shapes(train_pad_sequences_stem, train_encoded_labels, val_pad_sequences_stem, val_encoded_labels):
    print(f"Shape of train_pad_sequences_stem: {np.shape(train_pad_sequences_stem)}")
    print(f"Shape of train_encoded_labels: {np.shape(train_encoded_labels)}")
    print(f"Shape of val_pad_sequences_stem: {np.shape(val_pad_sequences_stem)}")
    print(f"Shape of val_encoded_labels: {np.shape(val_encoded_labels)}")
check_data_shapes(train_pad_sequences_stem, train_encoded_labels, val_pad_sequences_stem, val_encoded_labels)

Shape of train_pad_sequences_stem: (4078, 45)
Shape of train_encoded_labels: (4078, 22)
Shape of val_pad_sequences_stem: (900, 45)
Shape of val_encoded_labels: (900, 22)


In [67]:
# STEM 
embedding_dim = 100
vocab_size = len(stem_vocab) + 1  # Asegúrate de que stem_vocab es un diccionario bien formado

# Definición del modelo
model_stem = Sequential()
model_stem.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length_stem))  # Verifica max_sequence_length_stem
model_stem.add(GlobalMaxPooling1D())  # pooling layer
model_stem.add(Dense(64, activation='relu'))  # fully connected layer
model_stem.add(Dense(num_classes, activation='softmax'))  # output layer con softmax

# Compilar el modelo
model_stem.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
batch_size = 32
epochs = 2

model_stem.fit(train_pad_sequences_stem, train_encoded_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences_stem, val_encoded_labels))

# Evaluar el modelo en el conjunto de test
loss, accuracy = model_stem.evaluate(test_pad_sequences_stem, test_encoded_labels, batch_size=batch_size)
print(f"STEM Test accuracy: {accuracy:.2f}")


Epoch 1/2


c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6844 - loss: 1.9399 - val_accuracy: 0.7144 - val_loss: 0.8762
Epoch 2/2
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8121 - loss: 0.7134 - val_accuracy: 0.8756 - val_loss: 0.5614
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8606 - loss: 0.6419 
STEM Test accuracy: 0.83


In [68]:
# LEM 
embedding_dim = 100
vocab_size = len(lem_vocab) + 1

model_lem = Sequential()
model_lem.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim)) #layer 1 - embedding 
model_lem.add(GlobalMaxPooling1D()) #layer 2 - pooling
model_lem.add(Dense(64, activation='relu')) #layer 3 -relu
model_lem.add(Dense(num_classes, activation='softmax')) #layer 4 - softmax

# Compile the model
model_lem.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 2

model_lem.fit(train_pad_sequences_lem, train_encoded_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences_lem, val_encoded_labels))

# Evaluate the model on the test set
loss, accuracy = model_lem.evaluate(test_pad_sequences_lem, test_encoded_labels, batch_size=batch_size)
print(f"LEM Test accuracy: {accuracy:.2f}")

Epoch 1/2
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6776 - loss: 1.8661 - val_accuracy: 0.7500 - val_loss: 0.8851
Epoch 2/2
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8304 - loss: 0.6692 - val_accuracy: 0.8711 - val_loss: 0.5794


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 893
'y' sizes: 888


---

 <h1><a name="ex-five"><center> ✏ Exercici 5 ✏ </a></h1>

Modifiqueu els següents paràmetres del model anterior i analitzeu com afecten a la seva *accuracy*:

 1. **Preprocessament.** Modifiqueu el Tokenizer per canviar la mida del vostre vocabulari i afegiu nous passos de preprocessament. Alguns possibles canvis són canviar la mida del vocabulari, treure la capitalització o fer servir *lemmatització* o *stemming*.

 2. **Mida dels Embeddings.** Proveu diferents mides d'*Embeddings* i observeu com canvia l'*accuracy* del model. Heu d'explicar les vostres conclusions.

 3. **Xarxes Convolucionals.** Afegiu capes convolucionals al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació a l'hora d'escollir-los. Recordeu, que també podeu provar diferents configuracions de *pooling*.

 4. **Xarxes Recurrents.**  Afegiu capes recurrents al vostre model (LSTM, GRU). Expliqueu amb detall els valors que heu provat i la vostra motivació.

 5. **Regularització.** Quan proveu configuracions amb més paràmetres veureu que el model comença a tenir *overfitting* molt prompte durant l'entrenament. Afegiu *Dropout* al vostre model. Heu d'explicar la vostra decisió de valors i de posició dins de la xarxa.

 6. **Balancejat de les classes.** Si analitzeu el dataset, veureu que la freqüència de les classes està molt desbalancejada. Keras us permet afegir un pes per a cada classe a l'hora de calcular la loss (Mireu el paràmetre "class_weigth" a la documentació https://keras.io/api/models/model_training_apis/). Calculeu un pes per a cada classe i afegiu-lo al mètode fit del vostre model.

 ---

In [ ]:
# PROVAR de fer FOCAL LOSS


<h1><a name="section-four"> 4. Lliurable </a></h1>

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Experiments i Resultats (amb raonament).
3. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.

 ---